In [ ]:
import random
import networkx as nx
import numpy as np
import itertools
import argparse
import plotly.graph_objects as go
from typing import Dict, List

# Define the functions directly in this file
def calculate_loss(graph: nx.Graph, coloring: Dict[int, int]) -> int:
    """Loss function: counts number of adjacent vertices with same color"""
    conflicts = 0
    for u, v in graph.edges():
        if coloring[u] == coloring[v]:
            conflicts += 1
    return conflicts

def is_valid_coloring(graph: nx.Graph, coloring: Dict[int, int]) -> bool:
    """Check if coloring is valid (no adjacent vertices have same color)"""
    return calculate_loss(graph, coloring) == 0

# Algorytm próbkowania
def random_coloring(graph: nx.Graph, max_colors: int) -> Dict[int, int]:
    """Assign random colors to nodes, and if more than max_colors are used, reassign colors"""

    nodes = list(graph.nodes())
    total_attempts = 0
    
    while len(set(coloring.values())) <= max_colors:
        coloring = {node: random.randint(0, max_colors - 1) for node in nodes}
        total_attempts += 1
         
    return coloring, total_attempts
    
# Algorytm pełnego przeglądu
def brute_force_coloring(graph: nx.Graph, max_colors: int) -> tuple[Dict[int, int], List[int], int]:
    """Exhaustive search for minimum coloring"""
    nodes = list(graph.nodes())
    loss_history = []
    total_attempts = 0
    
    for coloring_tuple in itertools.product(range(max_colors), repeat=len(nodes)):
        total_attempts += 1
        coloring = dict(zip(nodes, coloring_tuple))
        loss = calculate_loss(graph, coloring)
        loss_history.append(loss)
        
        num_colors = len(set(coloring.values()))
        if num_colors <= max_colors:
            print(f"Found valid coloring with {num_colors} colors after {total_attempts} attempts")
            return coloring, loss_history, total_attempts
           
    # This line should never be reached since max_colors = n_nodes guarantees a solution
    print("Warning: No valid coloring found (this should not happen with max_colors = n_nodes)")
    return {}, loss_history, total_attempts

# Create sample graph
n_nodes = 20
G = nx.Graph()
coloring = {}

for node in range(n_nodes):
    color = random.randint(0, n_nodes)
    G.add_node(node)
    coloring[node] = color

# Add edges completely randomly (probability is adjustable)
edge_probability = 0.3  # Probability of adding an edge between two nodes
for i in range(n_nodes):
    for j in range(i + 1, n_nodes):  # Ensure no self-loops and no duplicate edges
        if random.random() < edge_probability:
            G.add_edge(i, j)


# Set max_colors to number of vertices
max_colors = 4

# coloring_result, loss_history, attempts = brute_force_coloring(G, max_colors)
# print("=== Brute Force Coloring Results ===")

coloring_result, attempts = random_coloring(G, max_colors)
# print(f"Loss_history: {loss_history}")
print(f"Colors: {max_colors} Attempts: {attempts}")

# Visualization
pos = {i: np.random.rand(3) for i in G.nodes()}
edge_x, edge_y, edge_z = [], [], []
for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

edge_trace = go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    line=dict(width=2, color="gray"),
    hoverinfo="none", mode="lines"
)

node_x, node_y, node_z, node_colors = [], [], [], []
for node in G.nodes():
    x, y, z = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)
    node_colors.append(coloring_result[node])  # Use the coloring result to color nodes

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode="markers",
    marker=dict(size=8, color=node_colors, colorscale="Jet", opacity=0.8)
)

fig = go.Figure(data=[edge_trace, node_trace])
fig.update_layout(
    title="3D Interactive Graph Coloring (Brute Force)",
    showlegend=False,
    scene=dict(
        xaxis=dict(visible=False),
        yaxis=dict(visible=False),
        zaxis=dict(visible=False)
    )
)
fig.show()



UnboundLocalError: cannot access local variable 'coloring' where it is not associated with a value